오늘의 구현순서

1) 데이터 준비 (배경과 사람거리가 있는 것으로다가~)
2)시멘틱 세그멘테이션으로 피사체와 배경을 분리 
3)블러링 기술로 배경을 흐리게 만들기 
4)피사체를 배경의 원래 위치로 합성 

In [2]:
#패키지 불러오기 

import cv2
import numpy as np
import os
from glob import glob
from os.path import join
import tarfile
import urllib # 웹에서 데이터를 다운받을 때 사용합니다. 

from matplotlib import pyplot as plt
import tensorflow as tf

In [11]:
# 이미지 저장하기 

#이미지 저장은 저장경로를 변수에 지정하고 그것이 이미지 파일이기에 cv2로 읽습니다. 
import os 
img_path = os.getenv('HOME') + '/aiffel/human_segmentation/images/market'
img_orig = cv2.imread(img_path)
print (img_orig.shape)
len(img_orig[0]) #가로 세로..음...가로 183개, 세로 275인 RGB파일 

(183, 275, 3)


275

# 세그멘테이션으로 사람 분리하기 
![image](https://user-images.githubusercontent.com/68840414/108790657-9b068b00-75c0-11eb-9a05-9b1dddbc115e.png)

* instance segmentation : 같은 클래스 분류 
* semantic segmentaion: 같은 클래스 분류 하지 않음

원본이미지 

![image](https://user-images.githubusercontent.com/68840414/108791202-dd7c9780-75c1-11eb-9b8d-8d45205028db.png)

인스턴스와 시멘틱 segmentation
![image](https://user-images.githubusercontent.com/68840414/108791183-d5bcf300-75c1-11eb-9a3f-3043059a7d1b.png)



